In [25]:
import llama_cpp
from llama_cpp import llama_supports_gpu_offload

print("Versión llama-cpp-python:", llama_cpp.__version__)
print("GPU offload soportado:", llama_supports_gpu_offload())


Versión llama-cpp-python: 0.3.16
GPU offload soportado: True


In [26]:
from __future__ import annotations

import json
import re
from typing import Any, Dict, List

import numpy as np
import fitz  # pymupdf
from sentence_transformers import SentenceTransformer
from llama_cpp import Llama
import torch
import copy


In [27]:
RUTA_MODELOS = "../ml-letter/models/"
MODEL_PATH = RUTA_MODELOS + "qwen2.5-3b-instruct-q5_k_m.gguf"

llm = Llama(
    model_path=MODEL_PATH,
    n_ctx=8192,          # contexto grande para políticas + explicación
    n_gpu_layers=20,
    n_batch=64,
    logits_all=False,
    n_threads=8,
    n_threads_batch=8,
    verbose=True,
)


llama_model_load_from_file_impl: using device CUDA0 (NVIDIA GeForce RTX 4050 Laptop GPU) - 3091 MiB free
llama_model_loader: loaded meta data with 26 key-value pairs and 435 tensors from ../ml-letter/models/qwen2.5-3b-instruct-q5_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = qwen2.5-3b-instruct
llama_model_loader: - kv   3:                            general.version str              = v0.1-v0.1
llama_model_loader: - kv   4:                           general.finetune str              = qwen2.5-3b-instruct
llama_model_loader: - kv   5:                         general.size_label str              = 3.4B
llama_model_loade

In [28]:
PDF_PATH = "../data/RAG/codigo_etica_inbursa.pdf"

doc = fitz.open(PDF_PATH)

pages_text = []
for page in doc:
    text = page.get_text()
    text = re.sub(r"\s+", " ", text)
    pages_text.append(text)

full_text = "\n".join(pages_text)

print("Longitud total del texto del PDF:", len(full_text))


Longitud total del texto del PDF: 35450


In [29]:
def chunk_text(text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap
    return chunks

policy_chunks = chunk_text(full_text)

embed_model = SentenceTransformer("all-MiniLM-L6-v2")
policy_embeddings = embed_model.encode(policy_chunks, show_progress_bar=True)


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 31b92f20-736f-405c-a64a-99aa08aeda11)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].
Batches: 100%|██████████| 3/3 [00:00<00:00, 15.29it/s]


In [30]:
def search_policy_context(query: str, top_k: int = 3) -> str:
    query_embedding = embed_model.encode([query])[0]
    scores = np.dot(policy_embeddings, query_embedding)
    top_indices = np.argsort(scores)[-top_k:][::-1]
    return "\n".join(policy_chunks[i] for i in top_indices)


In [31]:
# # Ejemplo de payload de entrada LOW RISK
# payload = {
#   "application": {
#     "id": "6976299ddb63aa4f0fb25cbb",
#     "probabilityDefault": 0.297548006180932,
#     "riskBand": "LOW",
#     "createdAt": "2026-01-25T14:33:01.469Z",
#     "amount": 5000,
#     "durationMonths": 24,
#     "topFeatures": [
#       {
#         "feature": "Purpose",
#         "feature_label": "Propósito del crédito",
#         "value_label": "Automóvil (nuevo)",
#         "impact_direction": "REDUCES_RISK",
#         "impact_text": "reduce el riesgo de incumplimiento"
#       },
#       {
#         "feature": "ForeignWorker",
#         "feature_label": "Trabajador extranjero",
#         "value_label": "No es trabajador extranjero",
#         "impact_direction": "INCREASES_RISK",
#         "impact_text": "incrementa el riesgo de incumplimiento"
#       },
#       {
#         "feature": "PersonalStatusSex",
#         "feature_label": "Estado civil y sexo",
#         "value_label": "Hombre: casado/viudo",
#         "impact_direction": "REDUCES_RISK",
#         "impact_text": "reduce el riesgo de incumplimiento"
#       },
#       {
#         "feature": "Status",
#         "feature_label": "Estado de la cuenta corriente",
#         "value_label": "Saldo < 0",
#         "impact_direction": "INCREASES_RISK",
#         "impact_text": "incrementa el riesgo de incumplimiento"
#       },
#       {
#         "feature": "OtherInstallmentPlans",
#         "feature_label": "Otros planes de pago",
#         "value_label": "Ninguno",
#         "impact_direction": "REDUCES_RISK",
#         "impact_text": "reduce el riesgo de incumplimiento"
#       }
#     ]
#   },
#   "customer": {
#     "id": "69761dcd0df92f4b9ef3aa1f",
#     "name": "nuevo pad",
#     "income": 0,
#     "status": "ACTIVE"
#   }
# }


In [32]:
# # Caso 1 – Perfil muy positivo (solo REDUCES_RISK)
# payload = {
#     "application": {
#         "id": "TEST-APP-APPROVE-001",
#         "probabilityDefault": 0.18,
#         "riskBand": "LOW",
#         "createdAt": "2026-01-26T10:00:00.000Z",
#         "amount": 8000,
#         "durationMonths": 18,
#         "topFeatures": [
#             {
#                 "feature": "CreditHistory",
#                 "feature_label": "Historial crediticio",
#                 "value_label": "Créditos anteriores pagados puntualmente",
#                 "impact_direction": "REDUCES_RISK",
#                 "impact_text": "reduce de forma importante el riesgo de incumplimiento"
#             },
#             {
#                 "feature": "Savings",
#                 "feature_label": "Cuenta de ahorros",
#                 "value_label": "Ahorros elevados",
#                 "impact_direction": "REDUCES_RISK",
#                 "impact_text": "proporciona respaldo de liquidez y reduce el riesgo"
#             },
#             {
#                 "feature": "EmploymentDuration",
#                 "feature_label": "Antigüedad laboral",
#                 "value_label": "Más de 5 años",
#                 "impact_direction": "REDUCES_RISK",
#                 "impact_text": "indica estabilidad en los ingresos y reduce el riesgo"
#             },
#             {
#                 "feature": "InstallmentRate",
#                 "feature_label": "Cuota sobre el ingreso",
#                 "value_label": "Menos del 20% del ingreso",
#                 "impact_direction": "REDUCES_RISK",
#                 "impact_text": "la carga de deuda es baja y reduce el riesgo"
#             },
#             {
#                 "feature": "CreditAmount",
#                 "feature_label": "Monto del crédito",
#                 "value_label": "Monto bajo",
#                 "impact_direction": "REDUCES_RISK",
#                 "impact_text": "el monto solicitado es manejable respecto al perfil"
#             }
#         ]
#     },
#     "customer": {
#         "id": "TEST-CUST-APPROVE-001",
#         "name": "Cliente Perfil Positivo",
#         "income": 28000,
#         "status": "ACTIVE"
#     }
# }


In [33]:
# Caso 2 – Perfil mixto / intermedio (riesgos + mitigantes) → REVIEW
payload = {
    "application": {
        "id": "TEST-APP-REVIEW-001",
        "probabilityDefault": 0.48,
        "riskBand": "MEDIUM",
        "createdAt": "2026-01-26T10:05:00.000Z",
        "amount": 20000,
        "durationMonths": 36,
        "topFeatures": [
            {
                "feature": "CreditHistory",
                "feature_label": "Historial crediticio",
                "value_label": "Algunos retrasos en el pasado",
                "impact_direction": "INCREASES_RISK",
                "impact_text": "incrementa el riesgo por incumplimientos previos"
            },
            {
                "feature": "Status",
                "feature_label": "Estado de la cuenta corriente",
                "value_label": "Saldo negativo reciente",
                "impact_direction": "INCREASES_RISK",
                "impact_text": "indica presión de liquidez y aumenta el riesgo"
            },
            {
                "feature": "InstallmentRate",
                "feature_label": "Cuota sobre el ingreso",
                "value_label": "Entre 25% y 35% del ingreso",
                "impact_direction": "INCREASES_RISK",
                "impact_text": "la carga de deuda es relevante respecto al ingreso"
            },
            {
                "feature": "Duration",
                "feature_label": "Duración del crédito",
                "value_label": "Plazo medio",
                "impact_direction": "INCREASES_RISK",
                "impact_text": "mayor plazo implica mayor exposición al riesgo"
            },
            {
                "feature": "Savings",
                "feature_label": "Cuenta de ahorros",
                "value_label": "Ahorros moderados",
                "impact_direction": "REDUCES_RISK",
                "impact_text": "ayuda a compensar parcialmente el riesgo"
            }
        ]
    },
    "customer": {
        "id": "TEST-CUST-REVIEW-001",
        "name": "Cliente Perfil Intermedio",
        "income": 23000,
        "status": "ACTIVE"
    }
}


In [34]:
# # Caso 3 – Perfil claramente riesgoso (solo INCREASES_RISK) → DECLINE
# payload = {
#     "application": {
#         "id": "TEST-APP-DECLINE-001",
#         "probabilityDefault": 0.85,
#         "riskBand": "HIGH",
#         "createdAt": "2026-01-26T10:10:00.000Z",
#         "amount": 45000,
#         "durationMonths": 48,
#         "topFeatures": [
#             {
#                 "feature": "CreditHistory",
#                 "feature_label": "Historial crediticio",
#                 "value_label": "Retrasos significativos en el pasado",
#                 "impact_direction": "INCREASES_RISK",
#                 "impact_text": "incrementa significativamente el riesgo de incumplimiento"
#             },
#             {
#                 "feature": "Savings",
#                 "feature_label": "Cuenta de ahorros",
#                 "value_label": "Sin ahorros registrados",
#                 "impact_direction": "INCREASES_RISK",
#                 "impact_text": "no se observa respaldo de liquidez"
#             },
#             {
#                 "feature": "NumberExistingCredits",
#                 "feature_label": "Número de créditos vigentes",
#                 "value_label": "Varios créditos activos",
#                 "impact_direction": "INCREASES_RISK",
#                 "impact_text": "indica un nivel elevado de endeudamiento"
#             },
#             {
#                 "feature": "CreditAmount",
#                 "feature_label": "Monto del crédito",
#                 "value_label": "Monto alto",
#                 "impact_direction": "INCREASES_RISK",
#                 "impact_text": "el monto solicitado es elevado respecto al perfil"
#             },
#             {
#                 "feature": "InstallmentRate",
#                 "feature_label": "Cuota sobre el ingreso",
#                 "value_label": "35% o más del ingreso",
#                 "impact_direction": "INCREASES_RISK",
#                 "impact_text": "la carga de deuda sería muy alta"
#             }
#         ]
#     },
#     "customer": {
#         "id": "TEST-CUST-DECLINE-001",
#         "name": "Cliente Perfil Riesgoso",
#         "income": 17000,
#         "status": "ACTIVE"
#     }
# }


In [35]:
SENSITIVE_FEATURES = {
    "ForeignWorker",
    "PersonalStatusSex",
    "Sex",
    "Age",
    "Race",
    "MaritalStatus",
}

def filter_sensitive_features(payload: Dict[str, Any]) -> None:
    app = payload.get("application", {})
    feats = app.get("topFeatures", [])
    app["topFeatures"] = [
        f for f in feats if f.get("feature") not in SENSITIVE_FEATURES
    ]
    payload["application"] = app

filter_sensitive_features(payload)

print(json.dumps(payload["application"]["topFeatures"], indent=2, ensure_ascii=False))


[
  {
    "feature": "CreditHistory",
    "feature_label": "Historial crediticio",
    "value_label": "Algunos retrasos en el pasado",
    "impact_direction": "INCREASES_RISK",
    "impact_text": "incrementa el riesgo por incumplimientos previos"
  },
  {
    "feature": "Status",
    "feature_label": "Estado de la cuenta corriente",
    "value_label": "Saldo negativo reciente",
    "impact_direction": "INCREASES_RISK",
    "impact_text": "indica presión de liquidez y aumenta el riesgo"
  },
  {
    "feature": "InstallmentRate",
    "feature_label": "Cuota sobre el ingreso",
    "value_label": "Entre 25% y 35% del ingreso",
    "impact_direction": "INCREASES_RISK",
    "impact_text": "la carga de deuda es relevante respecto al ingreso"
  },
  {
    "feature": "Duration",
    "feature_label": "Duración del crédito",
    "value_label": "Plazo medio",
    "impact_direction": "INCREASES_RISK",
    "impact_text": "mayor plazo implica mayor exposición al riesgo"
  },
  {
    "feature": "Savin

In [36]:
def infer_preliminary_status(payload: Dict[str, Any]) -> str:
    app = payload["application"]
    band = app["riskBand"]
    prob = app["probabilityDefault"]

    if band == "LOW" and prob < 0.30:
        return "APPROVE"
    if band == "MEDIUM" or (band == "LOW" and prob >= 0.30):
        return "REVIEW"
    return "DECLINE"

payload["decision"] = infer_preliminary_status(payload)
print("Estatus preliminar:", payload["decision"])


Estatus preliminar: REVIEW


In [37]:
policy_query = f"""
Evaluación preliminar de solicitudes de crédito.
Principios de no discriminación, trato digno, transparencia
y explicación de decisiones basada en criterios financieros.
"""

policy_context_text = search_policy_context(policy_query, top_k=3)


In [38]:
system_prompt = """
Eres un asistente de redacción para Grupo Financiero Inbursa.

Tu función es generar CARTAS DE EVALUACIÓN PRELIMINAR de solicitudes de crédito.

REGLAS OBLIGATORIAS:
- No decides la aprobación ni el rechazo definitivo.
- La decisión final siempre será comunicada por personal de Inbursa.
- El campo "decision" es un estatus preliminar y NO debe modificarse.
- Si en "topFeatures" existen factores con:
  - impact_direction = "REDUCES_RISK"
  - impact_direction = "INCREASES_RISK"
  debes SEPARARLOS en secciones distintas dentro de la carta.

- NUNCA incluyas un factor con "impact_direction = INCREASES_RISK"
  dentro de una sección que describa factores positivos,
  favorables o que "contribuyen positivamente".

- Usa exactamente estas etiquetas de sección cuando aplique:
  - "Factores que contribuyen positivamente a tu evaluación"
  - "Factores que podrían incrementar el nivel de riesgo"

- Si existe al menos un factor con "impact_direction = INCREASES_RISK",
  DEBES crear la sección de riesgo, incluso si la decisión es "APPROVE".
  
PROHIBICIONES ABSOLUTAS:
- No menciones ni utilices raza, edad, sexo, género, nacionalidad,
  condición de extranjero, estado civil u otros atributos protegidos.
- No uses esos factores aunque aparezcan en el JSON de entrada.

CUANDO USES topFeatures:
- Para cada factor que menciones en la carta o en "bullets", respeta SIEMPRE:
  - "feature_label"  (nombre del factor),
  - "value_label"    (situación observada),
  - "impact_text"    (cómo influye en el riesgo).
- Puedes reformular ligeramente el texto, pero NO debes contradecir el "value_label" ni el "impact_text".
- Un formato recomendado para las razones es:
  - "{feature_label}: {value_label}. {impact_text}".
- NO inventes situaciones contrarias a los datos. Por ejemplo, si el "value_label" dice "Saldo < 0",
  nunca debes decir que "no tiene saldo negativo" o algo equivalente.
  
REGLA SOBRE EL SENTIDO DEL IMPACTO:
- Si impact_direction = "REDUCES_RISK", debes usar expresiones coherentes como
  "reduce el riesgo", "disminuye el riesgo" o similares.
- Si impact_direction = "INCREASES_RISK", debes usar expresiones coherentes como
  "incrementa el riesgo", "aumenta el riesgo" o similares.
- NUNCA digas que un factor "reduce el riesgo" si en los datos aparece con
  impact_direction = "INCREASES_RISK", ni al revés.


CONTENIDO DE LA CARTA:
- Explica de forma preliminar factores que reducen o incrementan el riesgo.
- Usa solo información financiera permitida (monto, plazo, historial, etc.).
- Mantén un tono respetuoso, claro y no discriminatorio.
- Aclara siempre que la decisión es preliminar.

FORMATO DE SALIDA (EXACTO):
{
  "decision": "APPROVE" | "REVIEW" | "DECLINE",
  "letterSubject": "string",
  "letterText": "string",
  "bullets": ["string"],
  "safetyFlags": []
}

No incluyas texto fuera del JSON.
""".strip()


In [39]:
few_shot_examples = [
    {
    "input": {
        "application": {
            "id": "APP-0001-LOW-MIXED",
            "probabilityDefault": 0.28,
            "riskBand": "LOW",
            "createdAt": "2026-01-25T14:33:01.469Z",
            "amount": 5000,
            "durationMonths": 24,
            "topFeatures": [
                {
                    "feature": "Purpose",
                    "feature_label": "Propósito del crédito",
                    "value_label": "Automóvil (nuevo)",
                    "impact_direction": "REDUCES_RISK",
                    "impact_text": "reduce el riesgo de incumplimiento"
                },
                {
                    "feature": "Status",
                    "feature_label": "Estado de la cuenta corriente",
                    "value_label": "Saldo < 0",
                    "impact_direction": "INCREASES_RISK",
                    "impact_text": "incrementa el riesgo de incumplimiento"
                },
                {
                    "feature": "OtherInstallmentPlans",
                    "feature_label": "Otros planes de pago",
                    "value_label": "Ninguno",
                    "impact_direction": "REDUCES_RISK",
                    "impact_text": "reduce el riesgo de incumplimiento"
                }
            ]
        },
        "customer": {
            "id": "CUST-LOW-002",
            "name": "Ejemplo Perfil Mixto",
            "income": 24000,
            "status": "ACTIVE"
        },
        "decision": "APPROVE"
    },
    "output": {
        "decision": "APPROVE",
        "letterSubject": "Resultado preliminar de tu solicitud de crédito",
        "letterText": """Estimado(a) Ejemplo Perfil Mixto:

Con base en una evaluación preliminar de tu solicitud de crédito, se observa un perfil que, de manera inicial, resulta favorable. Esta evaluación se realiza conforme a nuestras políticas internas y a la normatividad aplicable, respetando los principios de honestidad, transparencia y trato digno hacia nuestros clientes.

De manera preliminar, se identifican los siguientes factores que contribuyen positivamente a tu evaluación:

- Propósito del crédito: Automóvil (nuevo). Reduce el riesgo de incumplimiento.
- Otros planes de pago: Ninguno. Reduce el riesgo de incumplimiento.

Asimismo, se identifica el siguiente factor que podría incrementar el nivel de riesgo y que será considerado en el análisis integral:

- Estado de la cuenta corriente: Saldo < 0. Incrementa el riesgo de incumplimiento.

Te recordamos que esta es una valoración preliminar basada en la información disponible. La decisión final sobre tu solicitud será analizada y comunicada por el personal de Grupo Financiero Inbursa a través de los canales oficiales.

Agradecemos la confianza que depositas en nosotros y reiteramos nuestro compromiso de brindarte un servicio claro, profesional y libre de cualquier tipo de discriminación.

Atentamente,
Área de Crédito
Grupo Financiero Inbursa""",
        "bullets": [
            "Propósito del crédito: Automóvil (nuevo). Reduce el riesgo de incumplimiento.",
            "Otros planes de pago: Ninguno. Reduce el riesgo de incumplimiento.",
            "Estado de la cuenta corriente: Saldo < 0. Incrementa el riesgo de incumplimiento."
        ],
        "safetyFlags": []
    }
},
    {
        "input": {
            "application": {
                "id": "APP-0002-MEDIUM",
                "probabilityDefault": 0.52,
                "riskBand": "MEDIUM",
                "createdAt": "2026-01-25T14:45:10.000Z",
                "amount": 15000,
                "durationMonths": 36,
                "topFeatures": [
                    {
                        "feature": "CreditHistory",
                        "feature_label": "Historial crediticio",
                        "value_label": "Algunos retrasos en el pasado",
                        "impact_direction": "INCREASES_RISK",
                        "impact_text": "incrementa el riesgo por incumplimientos previos"
                    },
                    {
                        "feature": "Status",
                        "feature_label": "Estado de la cuenta corriente",
                        "value_label": "Saldo negativo reciente",
                        "impact_direction": "INCREASES_RISK",
                        "impact_text": "indica presión de liquidez y aumenta el riesgo"
                    },
                    {
                        "feature": "InstallmentRate",
                        "feature_label": "Cuota sobre el ingreso",
                        "value_label": "Entre 25% y 35% del ingreso",
                        "impact_direction": "INCREASES_RISK",
                        "impact_text": "la carga de deuda es relevante respecto al ingreso"
                    },
                    {
                        "feature": "Duration",
                        "feature_label": "Duración del crédito",
                        "value_label": "Plazo medio",
                        "impact_direction": "INCREASES_RISK",
                        "impact_text": "mayor plazo implica mayor exposición al riesgo"
                    },
                    {
                        "feature": "Savings",
                        "feature_label": "Cuenta de ahorros",
                        "value_label": "Ahorros moderados",
                        "impact_direction": "REDUCES_RISK",
                        "impact_text": "ayuda a compensar parcialmente el riesgo"
                    }
                ]
            },
            "customer": {
                "id": "CUST-MED-001",
                "name": "María López",
                "income": 22000,
                "status": "ACTIVE"
            },
            "decision": "REVIEW"
        },
        "output": {
            "decision": "REVIEW",
            "letterSubject": "Tu solicitud de crédito está en revisión preliminar",
            "letterText": """Estimado(a) María López:

Hemos realizado una evaluación preliminar de la información proporcionada en tu solicitud de crédito. Esta revisión se lleva a cabo en apego a nuestras políticas internas, a la regulación aplicable y a los principios de claridad, responsabilidad y respeto hacia nuestros clientes.

De manera inicial, se identifican los siguientes factores que incrementan el riesgo en tu evaluación:

- Historial crediticio: Algunos retrasos en el pasado. Esto incrementa el riesgo por incumplimientos previos.
- Estado de la cuenta corriente: Saldo negativo reciente. Esto indica presión de liquidez y aumenta el riesgo.
- Cuota sobre el ingreso: Entre 25% y 35% del ingreso. La carga de deuda es relevante respecto a tu ingreso.
- Duración del crédito: Plazo medio. Un mayor plazo implica una mayor exposición al riesgo a lo largo del tiempo.

Asimismo, se identifican los siguientes factores que ayudan a mitigar parcialmente el riesgo:

- Cuenta de ahorros: Ahorros moderados. Esto ayuda a compensar parcialmente el riesgo identificado.

Con base en lo anterior, tu solicitud se encuentra en revisión adicional. Es posible que nos pongamos en contacto contigo para solicitar información complementaria o para proponerte ajustes de monto o plazo que resulten más adecuados y sostenibles para ti.

Te recordamos que esta es una evaluación preliminar y que la decisión final sobre tu solicitud será comunicada por el personal de Grupo Financiero Inbursa a través de los canales oficiales.

Agradecemos tu comprensión y la confianza depositada en nosotros. Este proceso se realiza con criterios objetivos, sin discriminación y respetando en todo momento la confidencialidad de tu información.

Atentamente,
Área de Crédito
Grupo Financiero Inbursa""",
            "bullets": [
                "Factores que incrementan el riesgo: retrasos previos en el historial crediticio.",
                "Factores que incrementan el riesgo: saldo negativo reciente en la cuenta corriente.",
                "Factores que incrementan el riesgo: cuota mensual estimada entre 25% y 35% del ingreso.",
                "Factores que incrementan el riesgo: plazo medio que incrementa la exposición al riesgo.",
                "Factor que mitiga parcialmente el riesgo: cuenta de ahorros con ahorros moderados."
            ],
            "safetyFlags": []
        }
    },
    {
        "input": {
            "application": {
                "id": "APP-0003-HIGH",
                "probabilityDefault": 0.83,
                "riskBand": "HIGH",
                "createdAt": "2026-01-25T15:05:30.000Z",
                "amount": 40000,
                "durationMonths": 48,
                "topFeatures": [
                    {
                        "feature": "CreditHistory",
                        "feature_label": "Historial crediticio",
                        "value_label": "Retrasos significativos en el pasado",
                        "impact_direction": "INCREASES_RISK",
                        "impact_text": "incrementa significativamente el riesgo de incumplimiento"
                    },
                    {
                        "feature": "Savings",
                        "feature_label": "Cuenta de ahorros",
                        "value_label": "Sin ahorros registrados",
                        "impact_direction": "INCREASES_RISK",
                        "impact_text": "no se observa respaldo de liquidez"
                    },
                    {
                        "feature": "NumberExistingCredits",
                        "feature_label": "Número de créditos vigentes",
                        "value_label": "Varios créditos activos",
                        "impact_direction": "INCREASES_RISK",
                        "impact_text": "indica un nivel elevado de endeudamiento"
                    },
                    {
                        "feature": "CreditAmount",
                        "feature_label": "Monto del crédito",
                        "value_label": "Monto alto",
                        "impact_direction": "INCREASES_RISK",
                        "impact_text": "el monto solicitado es elevado respecto al perfil"
                    },
                    {
                        "feature": "InstallmentRate",
                        "feature_label": "Cuota sobre el ingreso",
                        "value_label": "35% o más del ingreso",
                        "impact_direction": "INCREASES_RISK",
                        "impact_text": "la carga de deuda sería muy alta"
                    }
                ]
            },
            "customer": {
                "id": "CUST-HIGH-001",
                "name": "Carlos Ramírez",
                "income": 18000,
                "status": "ACTIVE"
            },
            "decision": "DECLINE"
        },
        "output": {
            "decision": "DECLINE",
            "letterSubject": "Resultado preliminar de la evaluación de tu solicitud de crédito",
            "letterText": """Estimado(a) Carlos Ramírez:

Agradecemos el interés que has mostrado en nuestros productos y el tiempo dedicado a presentar tu solicitud de crédito.

Con base en un análisis preliminar realizado conforme a nuestras políticas internas, a la regulación vigente y a los principios de honestidad, transparencia y no discriminación, se han identificado diversos factores que podrían dificultar la aprobación de tu solicitud:

De manera preliminar, se identifican los siguientes factores que incrementan de forma importante el riesgo en tu evaluación:

- Historial crediticio: Retrasos significativos en el pasado. Esto incrementa significativamente el riesgo de incumplimiento.
- Cuenta de ahorros: Sin ahorros registrados. No se observa respaldo de liquidez para nuevas obligaciones.
- Número de créditos vigentes: Varios créditos activos. Esto indica un nivel elevado de endeudamiento.
- Monto del crédito: Monto alto. El monto solicitado es elevado respecto a tu perfil financiero.
- Cuota sobre el ingreso: 35% o más del ingreso. La carga de deuda sería muy alta y podría comprometer tu capacidad de pago.

En esta evaluación preliminar no se han identificado factores suficientes que compensen estos riesgos, por lo que, de forma inicial, se aprecia un perfil con un nivel de riesgo elevado para la aprobación del crédito.

Esta evaluación es preliminar y no constituye, por sí misma, una decisión definitiva. La determinación final sobre tu solicitud será tomada y comunicada por el personal de Grupo Financiero Inbursa, considerando la información disponible y, en su caso, la documentación adicional que se integre al expediente.

Te invitamos a considerar alternativas para mejorar tu situación financiera, como reducir parte de tu endeudamiento actual, fortalecer tu ahorro y procurar un historial de pagos puntual. En el futuro podrás presentar una nueva solicitud, la cual será evaluada de acuerdo con las condiciones que presentes en ese momento.

Reiteramos nuestro compromiso de brindarte siempre un trato respetuoso, equitativo y confidencial, acorde con nuestro Código de Ética.

Atentamente,
Área de Crédito
Grupo Financiero Inbursa""",
            "bullets": [
                "Historial crediticio: retrasos significativos en el pasado. Incrementa significativamente el riesgo de incumplimiento.",
                "Cuenta de ahorros: sin ahorros registrados. No se observa respaldo de liquidez.",
                "Número de créditos vigentes: varios créditos activos. Indica un nivel elevado de endeudamiento.",
                "Monto del crédito: monto alto. El monto solicitado es elevado respecto al perfil financiero.",
                "Cuota sobre el ingreso: 35% o más del ingreso. La carga de deuda sería muy alta y podría comprometer la capacidad de pago."
            ],
            "safetyFlags": []
        }
    }
]

print("Se han definido", len(few_shot_examples), "ejemplos de few-shot.")


Se han definido 3 ejemplos de few-shot.


In [40]:
def format_few_shot_block(few_shot_examples):
    """
    Convierte la lista de ejemplos en un bloque de texto:
    ### EJEMPLO N
    JSON_INPUT:
    {...}
    JSON_OUTPUT:
    {...}
    """
    blocks = []
    for i, ex in enumerate(few_shot_examples, start=1):
        inp = json.dumps(ex["input"], ensure_ascii=False, indent=2)
        out = json.dumps(ex["output"], ensure_ascii=False, indent=2)
        block = f"""### EJEMPLO {i}

JSON_INPUT:
{inp}

JSON_OUTPUT:
{out}
"""
        blocks.append(block)
    return "\n\n".join(blocks)

few_shot_subset = few_shot_examples  # o few_shot_examples[:2]

few_shot_text = format_few_shot_block(few_shot_subset)

print(f"Longitud de few_shot_text: {len(few_shot_text)} caracteres")


Longitud de few_shot_text: 11681 caracteres


In [41]:
payload_json_str = json.dumps(payload, ensure_ascii=False)

# Limitar tamaño del contexto de políticas
MAX_POLICY_CHARS = 6000
policy_context_for_prompt = policy_context_text[:MAX_POLICY_CHARS]

user_prompt = f"""
A continuación tienes información de contexto sobre el Código de Ética y las políticas internas del banco.
Debes respetar este contexto: no inventes artículos, cláusulas ni obligaciones que no estén aquí.

POLICY_CONTEXT:
\"\"\"
{policy_context_for_prompt}
\"\"\"

A continuación tienes algunos ejemplos de cómo, dado un JSON de entrada (JSON_INPUT),
se debe generar un JSON de salida (JSON_OUTPUT) con la carta preliminar correspondiente:

FEW_SHOT_EXAMPLES:
\"\"\"
{few_shot_text}
\"\"\"

Ahora tienes el JSON con la información de la solicitud de crédito actual
y el estatus PRELIMINAR ya calculado por el banco en el campo "decision"
(APPROVE, REVIEW o DECLINE). Este valor NO debe cambiarse.

JSON_INPUT:
{payload_json_str}

Instrucciones clave para esta tarea:
- Usa el POLICY_CONTEXT solo como referencia de tono ético y principios generales.
- Imita el estilo de los FEW_SHOT_EXAMPLES en estructura y nivel de detalle.
- Explica la situación como una EVALUACIÓN PRELIMINAR, no como una decisión definitiva.
- Indica que la decisión final será comunicada por el personal de Inbursa.
- Genera la carta y el JSON de salida siguiendo estrictamente las reglas
  y el formato definidos en el mensaje del sistema.
""".strip()

print(user_prompt[:1500], "...")


A continuación tienes información de contexto sobre el Código de Ética y las políticas internas del banco.
Debes respetar este contexto: no inventes artículos, cláusulas ni obligaciones que no estén aquí.

POLICY_CONTEXT:
"""
de los Clientes. vi. Comprometer los medios electrónicos empleados por los Clientes con el objetivo de instalar un código malicioso capaz de alterar la realización de operaciones con las Sociedades. vii. Acceder a los equipos tecnológicos de las Sociedades para modificar su funcionamiento con la finalidad de obtener información confidencial, recursos económicos, o alterar su operatividad viii. Alterar cheques y emitir cheques falsos. ix. Adquirir, poseer, utilizar tarjetas de crédito, débito, ch
, poseer, utilizar tarjetas de crédito, débito, cheques, esqueletos o formatos, o cualquier otro instrumento de pago en forma ilegítima o sin consentimiento de su titular. x. Acceder ilícitamente a los sistemas informáticos con la intención de modificar, destruir o copiar 

In [42]:
def call_llm(system_prompt: str, user_prompt: str, max_tokens: int = 768) -> str:
    full_prompt = (
        f"<system>\n{system_prompt}\n</system>\n"
        f"<user>\n{user_prompt}\n</user>\n"
        f"<assistant>\n"
    )

    output = llm(
        full_prompt,
        max_tokens=max_tokens,
        temperature=0.2,
        top_p=0.9,
        stop=["</assistant>"],
    )

    return output["choices"][0]["text"]


In [43]:
def extract_json(text: str) -> Dict[str, Any] | None:
    try:
        start = text.index("{")
        end = text.rindex("}") + 1
        return json.loads(text[start:end])
    except Exception as e:
        print("Error parseando JSON:", e)
        return None


In [44]:
def validate_letter_structure(letter: dict, payload: dict) -> dict:
    """
    Valida que la carta respete la estructura:
    - Si hay factores REDUCES_RISK e INCREASES_RISK, se mencionen positivos y riesgos.
    - Si hay al menos un INCREASES_RISK en el payload, debe haber una parte del texto
      que hable de factores que incrementan el nivel de riesgo.
    - Los factores de riesgo no deben aparecer en la sección positiva.
    Añade flags a safetyFlags según corresponda.
    """
    safety_flags = letter.get("safetyFlags", []) or []
    text = letter.get("letterText", "") or ""
    text_lower = text.lower()

    # Frase clave para la sección positiva (case-insensitive)
    POS_KEY = "factores que contribuyen positivamente a tu evaluación"

    # Varias frases posibles para identificar la sección de riesgo
    RISK_KEYS = [
        "podría incrementar el nivel de riesgo",
        "podrían incrementar el nivel de riesgo",
        "factores que podrían incrementar el nivel de riesgo",
        "factores que incrementan de forma importante el riesgo",
        "factores que incrementan el riesgo",
    ]

    # 1) Verificar si en el payload hay factores positivos y de riesgo
    top_features = payload.get("application", {}).get("topFeatures", [])
    has_risk_features = any(
        f.get("impact_direction") == "INCREASES_RISK" for f in top_features
    )
    has_positive_features = any(
        f.get("impact_direction") == "REDUCES_RISK" for f in top_features
    )

    # 2) ¿Existe la "sección positiva"?
    has_pos_section = POS_KEY in text_lower

    # 3) Buscar sección de riesgo por cualquiera de las frases clave
    risk_indices = [text_lower.index(k) for k in RISK_KEYS if k in text_lower]
    has_risk_section = len(risk_indices) > 0

    # 4) Construir secciones aproximadas para buscar violaciones
    positives_section = ""
    risk_section = ""

    if has_pos_section:
        start_pos = text_lower.index(POS_KEY) + len(POS_KEY)
        if has_risk_section:
            end_pos = min(risk_indices)
        else:
            end_pos = len(text_lower)
        positives_section = text_lower[start_pos:end_pos]

    if has_risk_section:
        start_risk = min(risk_indices) + len("factores")  # solo para separar un poco
        risk_section = text_lower[start_risk:]

    # 5) Si hay factores de riesgo en el payload pero no se ve ninguna frase de riesgo en la carta
    if has_risk_features and not has_risk_section:
        safety_flags.append("MISSING_RISK_SECTION")

    # 6) Si hay factores positivos pero no se ve la frase de sección positiva en la carta
    if has_positive_features and not has_pos_section:
        safety_flags.append("MISSING_POSITIVE_SECTION")

    # 7) Verificar que ningún factor INCREASES_RISK aparezca en la sección positiva
    for f in top_features:
        if f.get("impact_direction") == "INCREASES_RISK":
            feature_label = (f.get("feature_label") or "").lower()
            value_label = (f.get("value_label") or "").lower()
            if positives_section:
                if feature_label and feature_label in positives_section:
                    safety_flags.append("RISK_FEATURE_IN_POSITIVE_SECTION")
                if value_label and value_label in positives_section:
                    safety_flags.append("RISK_VALUE_IN_POSITIVE_SECTION")

    # Dejar sin duplicados
    if safety_flags:
        letter["safetyFlags"] = sorted(set(safety_flags))
    else:
        letter["safetyFlags"] = []

    return letter


In [45]:
def sanitize_letter_remove_risk_features(letter: dict, payload: dict) -> tuple[dict, dict]:
    """
    A partir de una carta generada por el modelo y el payload original:
    - Elimina del texto y de los bullets las menciones a factores con impact_direction = INCREASES_RISK.
    - Devuelve:
        - letter_sanitized: carta modificada.
        - payload_sanitized: payload con topFeatures filtrados para dejar solo REDUCES_RISK.
    Además añade un safetyFlag: "RISK_FACTORS_OMITTED_FROM_LETTER".
    """
    letter = copy.deepcopy(letter)
    payload_sanitized = copy.deepcopy(payload)

    top_features = payload.get("application", {}).get("topFeatures", []) or []
    risk_features = [f for f in top_features if f.get("impact_direction") == "INCREASES_RISK"]

    if not risk_features:
        # Nada que sanitizar, retornamos tal cual
        return letter, payload_sanitized

    # Función auxiliar para saber si una línea menciona un factor de riesgo
    def line_mentions_risk(line: str) -> bool:
        l = line.lower()
        for f in risk_features:
            fl = (f.get("feature_label") or "").lower()
            vl = (f.get("value_label") or "").lower()
            if (fl and fl in l) or (vl and vl in l):
                return True
        return False

    # 1) Limpiar letterText
    text = letter.get("letterText", "") or ""
    lines = text.split("\n")
    new_lines = [line for line in lines if not line_mentions_risk(line)]
    letter["letterText"] = "\n".join(new_lines)

    # 2) Limpiar bullets
    bullets = letter.get("bullets", []) or []
    new_bullets = []
    for b in bullets:
        if line_mentions_risk(b):
            continue
        new_bullets.append(b)
    letter["bullets"] = new_bullets

    # 3) Filtrar topFeatures en el payload sanitizado (dejar solo REDUCES_RISK)
    payload_sanitized["application"]["topFeatures"] = [
        f for f in top_features if f.get("impact_direction") == "REDUCES_RISK"
    ]

    # 4) Añadir safetyFlag informativo
    safety_flags = letter.get("safetyFlags", []) or []
    safety_flags.append("RISK_FACTORS_OMITTED_FROM_LETTER")
    letter["safetyFlags"] = sorted(set(safety_flags))

    return letter, payload_sanitized


In [46]:
def build_generic_review_letter(payload: dict, extra_flags: list[str] | None = None) -> dict:
    """
    Construye una carta genérica de revisión cuando el modelo no genera
    una carta segura después de reintentar.
    """
    customer = payload.get("customer", {}) or {}
    name = customer.get("name") or "Cliente"
    decision_fallback = "REVIEW"  # Carta genérica siempre en tono de revisión

    subject = "Tu solicitud de crédito está en revisión"

    text = f"""Estimado(a) {name}:

Hemos recibido tu solicitud de crédito y, por el momento, se encuentra en revisión adicional.

Esta evaluación se realiza conforme a nuestras políticas internas y a la normatividad aplicable, respetando en todo momento los principios de honestidad, transparencia y trato digno hacia nuestros clientes.

Tu solicitud será analizada con detenimiento por nuestro personal especializado. Una vez concluido el análisis, nos pondremos en contacto contigo a la brevedad para informarte el resultado o, en su caso, solicitarte información adicional que pueda ser necesaria.

Te recordamos que esta comunicación tiene un carácter preliminar y no constituye, por sí misma, una decisión definitiva sobre tu solicitud.

Agradecemos la confianza que depositas en Grupo Financiero Inbursa y reiteramos nuestro compromiso de brindarte un servicio claro, profesional y libre de cualquier tipo de discriminación.

Atentamente,
Área de Crédito
Grupo Financiero Inbursa"""

    safety_flags = extra_flags or []
    safety_flags.append("FALLBACK_GENERIC_REVIEW")

    return {
        "decision": decision_fallback,
        "letterSubject": subject,
        "letterText": text,
        "bullets": [
            "Tu solicitud se encuentra en revisión adicional.",
            "Tu caso será analizado con detenimiento por personal especializado.",
            "Nos pondremos en contacto contigo para informarte el resultado o solicitar información adicional."
        ],
        "safetyFlags": sorted(set(safety_flags)),
    }

In [47]:
def count_tokens(text: str) -> int:
    return len(llm.tokenize(text.encode("utf-8")))

prompt_full = (
    f"<system>\n{system_prompt}\n</system>\n"
    f"<user>\n{user_prompt}\n</user>\n"
    f"<assistant>\n"
)

print("TOKENS prompt_full:", count_tokens(prompt_full))

TOKENS prompt_full: 5166


In [48]:
max_tokens = 768
all_attempt_flags: list[str] = []
final_letter = None

print("\n===== INTENTO 1 (LLM) =====\n")

# 1) Primer intento: llamada normal al modelo con el payload completo
raw_output = call_llm(system_prompt, user_prompt, max_tokens=max_tokens)
print("RAW OUTPUT:\n", raw_output, "\n\n")

letter = extract_json(raw_output)

if letter is None:
    print("[WARN] No se pudo extraer un JSON válido en el primer intento.")
    all_attempt_flags.append("PARSE_ERROR")
else:
    # 1.a) Forzar que la decisión no cambie
    expected_decision = payload.get("decision")
    safety_flags = letter.get("safetyFlags", []) or []

    if expected_decision and letter.get("decision") != expected_decision:
        print("[WARN] El modelo cambió la decisión. Corrigiendo a la decisión del backend.")
        letter["decision"] = expected_decision
        safety_flags.append("DECISION_CORRECTED")

    letter["safetyFlags"] = safety_flags

    # 1.b) Validar estructura de la carta vs reglas de negocio (con payload completo)
    letter = validate_letter_structure(letter, payload)

    if not letter.get("safetyFlags"):
        # Carta limpia, la usamos tal cual
        final_letter = letter
        print("[INFO] Carta generada sin safetyFlags en el intento 1.")
    else:
        print("[WARN] safetyFlags presentes en el intento 1:", letter["safetyFlags"])
        all_attempt_flags.extend(letter["safetyFlags"])

        # 2) Segundo intento: sanitizar carta (quitar factores de riesgo problemáticos)
        print("\n===== INTENTO 2 (SANITIZAR CARTA) =====\n")
        letter_sanitized, payload_sanitized = sanitize_letter_remove_risk_features(letter, payload)

        # Validar de nuevo, pero ahora contra el payload sanitizado (sin INCREASES_RISK)
        letter_sanitized = validate_letter_structure(letter_sanitized, payload_sanitized)

        if not letter_sanitized.get("safetyFlags"):
            final_letter = letter_sanitized
            print("[INFO] Carta sanitizada sin safetyFlags. Se utilizará esta versión.")
        else:
            print("[WARN] safetyFlags presentes incluso después de sanitizar:", letter_sanitized["safetyFlags"])
            all_attempt_flags.extend(letter_sanitized["safetyFlags"])

# 3) Si después de todo esto no tenemos carta segura, usamos carta genérica
if final_letter is None:
    print("[INFO] No se obtuvo una carta segura después de los intentos. Usando carta genérica de revisión.")
    final_letter = build_generic_review_letter(payload, extra_flags=all_attempt_flags)

print("\n=== CARTA FINAL (VALIDADA / CON FALLBACK SI APLICÓ) ===")
print(json.dumps(final_letter, indent=2, ensure_ascii=False))

print("\n=== TEXTO DE LA CARTA ===\n")
print(final_letter.get("letterText", ""))



===== INTENTO 1 (LLM) =====



llama_perf_context_print:        load time =   16946.25 ms
llama_perf_context_print: prompt eval time =   16945.98 ms /  5166 tokens (    3.28 ms per token,   304.85 tokens per second)
llama_perf_context_print:        eval time =   22660.77 ms /   474 runs   (   47.81 ms per token,    20.92 tokens per second)
llama_perf_context_print:       total time =   40123.09 ms /  5640 tokens
llama_perf_context_print:    graphs reused =        458


RAW OUTPUT:
 {
  "decision": "REVIEW",
  "letterSubject": "Resultado preliminar de tu solicitud de crédito",
  "letterText": "Estimado(a) Cliente Perfil Intermedio:\n\nCon base en una evaluación preliminar de tu solicitud de crédito, se observa un perfil que, de manera inicial, resulta favorable. Esta evaluación se realiza conforme a nuestras políticas internas y a la normatividad aplicable, respetando los principios de honestidad, transparencia y trato digno hacia nuestros clientes.\n\nDe manera preliminar, se identifican los siguientes factores que contribuyen positivamente a tu evaluación:\n\n- Historial crediticio: Algunos retrasos en el pasado. Reduce el riesgo de incumplimiento.\n- Cuenta de ahorros: Ahorros moderados. Ayuda a compensar parcialmente el riesgo.\n\nAsimismo, se identifica el siguiente factor que podría incrementar el nivel de riesgo y que será considerado en el análisis integral:\n\n- Estado de la cuenta corriente: Saldo negativo reciente. Incrementa el riesgo de i

In [49]:
# raw_output = call_llm(system_prompt, user_prompt, max_tokens=768)
# print("RAW OUTPUT:\n", raw_output, "\n\n")

# letter = extract_json(raw_output)

# if letter:
#     # 1) Forzar que la decisión no cambie
#     expected_decision = payload.get("decision")
#     safety_flags = letter.get("safetyFlags", []) or []

#     if expected_decision and letter.get("decision") != expected_decision:
#         print("[WARN] El modelo cambió la decisión. Corrigiendo a la decisión del backend.")
#         letter["decision"] = expected_decision
#         safety_flags.append("DECISION_CORRECTED")

#     letter["safetyFlags"] = safety_flags

#     # 2) Validar estructura de la carta vs reglas de negocio
#     letter = validate_letter_structure(letter, payload)

#     print("=== CARTA FINAL (VALIDADA) ===")
#     print(json.dumps(letter, indent=2, ensure_ascii=False))

#     print("\n=== TEXTO DE LA CARTA ===\n")
#     print(letter.get("letterText", ""))
# else:
#     print("No se pudo extraer un JSON válido de la salida del modelo.")
